# Import Library

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

d:\Project\Final Project Indonesia AI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# !pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [16]:
# !pip install accelerate[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# Sentiment Analysist Use Pretrained Model BERT Uncased + Fine tuned using row dataset


In [2]:
df = pd.read_excel('lowercase_labels_datasets.xlsx')

In [4]:
df
print(df.columns)

Index(['text', 'sentiment'], dtype='object')


## pre processing:
- drop N/A

In [5]:
df.head()

,text,sentiment
0,deserved candidate promoted promptly unbiased ...,1.0
1,got lot learning platform monthly learning pla...,1.0
2,based business unit get experience company pol...,1.0
3,client project good use latest tech work,1.0
4,worked ibm year tc year rd company year comple...,1.0


In [7]:
# Remove rows with missing or invalid text data
df.dropna(subset=['text', 'sentiment'], inplace=True)


In [10]:
len(df)

46416

## preparation for sentiment analysis, consists:
- encode labels
- split data
- tokenize the text column on train and val

In [9]:
# Encode sentiment labels
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [10]:
# Split data into train and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2)


In [11]:
# Debugging: Print first few entries
print("First 5 train texts:", train_texts[:5])
print("First 5 train labels:", train_labels[:5])
print("First 5 val texts:", val_texts[:5])
print("First 5 val labels:", val_labels[:5])

First 5 train texts: ['as of now nothing from my end because everything going smooth for me.', 'nothing to like here', 'connect with other team to due team work', 'everything', 'just the compensation part']
First 5 train labels: [0, 1, 1, 1, 0]
First 5 val texts: ['bad behavior in bench time, work life balance is total on client wish for good or bad environment, appraisal is poor and in some project senior are doing micro management.', 'work atmosphere', 'now a days the way of handling projects become too difficult.', 'the best environment to work and very understanding manager . work flexibility is there and many more.', 'best work culture']
First 5 val labels: [0, 1, 0, 1, 1]


In [12]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 classes: negative and positive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Tokenize data
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128)
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=128)


In [14]:
# Create torch dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [15]:
!pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


## fine tuned, results, and save.

The fine tuned consists:
- epoch: 3
-  bath size: 16

In [16]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [17]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [18]:
# Train model
trainer.train()

Step,Training Loss
10,0.722500
20,0.736300
30,0.718100
40,0.703800
50,0.714700
60,0.689500
70,0.687900
80,0.666800
90,0.668100
100,0.651800


TrainOutput(global_step=7032, training_loss=0.5734468095080845, metrics={'train_runtime': 2592.8517, 'train_samples_per_second': 43.38, 'train_steps_per_second': 2.712, 'total_flos': 7398617098959360.0, 'train_loss': 0.5734468095080845, 'epoch': 3.0})

In [19]:
# Evaluate model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6024132966995239, 'eval_runtime': 66.7705, 'eval_samples_per_second': 140.391, 'eval_steps_per_second': 8.776, 'epoch': 3.0}


In [35]:
#import matplotlib.pyplot as plt

# Get the training logs
# train_logs = trainer.callback_handler.log_history

# Extract the training loss from the logs
# train_loss = [log["loss"] for log in train_logs if "loss" in log]

# Plot the training loss
# plt.plot(train_loss, label="Training Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("Training Loss Over Epochs")
# plt.legend()
# plt.show()

AttributeError: 'CallbackHandler' object has no attribute 'log_history'

In [20]:
# Save model pretrained model for hugging face
model.save_pretrained("D:\Project\Final Project Indonesia AI/01_09062024_bert_sentiment_model", from_pt=True)

In [ ]:
model.save_model('D:\Project\Final Project Indonesia AI')

In [ ]:
!pip install

## testing the fine tuned models!

In [5]:
# Load tokenizer and fine-tuned model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/AI FOR INDONESIA/Final Project/01_09062024_bert_sentiment_model") # the fine tuned model save

# Function to tokenize text and obtain BERT embeddings
def tokenize_and_predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits
    return predictions

# Function to compute percentage
def compute_percentage(predictions):
    softmax_output = torch.softmax(predictions, dim=1)
    positive_percentage = softmax_output[:, 1].item() * 100
    negative_percentage = 100 - positive_percentage
    return positive_percentage, negative_percentage

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [23]:
# User input
user_text = input("")

# Tokenize user input and get predictions
user_predictions = tokenize_and_predict(user_text)
positive_percentage, negative_percentage = compute_percentage(user_predictions)

# Determine sentiment
sentiment = "Positive" if user_predictions.argmax() == 1 else "Negative"

print("Sentiment:", sentiment)
print("Positive sentiment percentage:", positive_percentage)
print("Negative sentiment percentage:", negative_percentage)

🤯jobdesk
Sentiment: Negative
Positive sentiment percentage: 46.032196283340454
Negative sentiment percentage: 53.967803716659546


# Sentimen Analysist Use pretrained Model

In [45]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm



In [46]:
# Load dataset
data = pd.read_excel('/content/drive/MyDrive/AI FOR INDONESIA/Final Project/lowercase_text_only_datasets.xlsx')



In [47]:
data.head()

,text
0,deserved candidates are promoted promptly.\nun...
1,you got lot of learning platform and monthly l...
2,based on which business unit you are you will ...
3,some client projects are good as they use the ...
4,"i have worked in ibm (4 years) ,tcs (1 year) ,..."


In [48]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")



In [49]:
# Function to tokenize text and obtain BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # Average pooling over token embeddings



In [ ]:
# Apply BERT analysis to each text in the dataset
output_embeddings = []
for text in tqdm(data['text'].astype(str)):
    embeddings = get_bert_embeddings(text)
    output_embeddings.append(embeddings)



 92%|█████████▏| 49949/54026 [1:34:57<06:43, 10.10it/s]

In [1]:
# Add the BERT analysis results to the dataset as a new column
data['output_Bert'] = output_embeddings

# Save the updated dataset
data.to_csv('output_dataset.csv', index=False)

NameError: name 'output_embeddings' is not defined

# Sentiment Analysist Use Pretrained Model BERT Uncased + Fine tuned from cleaned datasets  


In [3]:
df = pd.read_excel('preprocessing_capegini.xlsx')

In [4]:
df

,text,sentiment
0,deserved candidate promoted promptly unbiased ...,1.0
1,got lot learning platform monthly learning pla...,1.0
2,based business unit get experience company pol...,1.0
3,client project good use latest tech work,1.0
4,worked ibm year tc year rd company year comple...,1.0
...,...,...
54022,amazon adopted giving responsibility workplace...,0.0
54023,terrible experience working amazon pay leaders...,0.0
54024,logistics supply management application operat...,0.0
54025,toxic culture depend team hierarchical questio...,0.0


In [5]:
# Remove rows with missing or invalid text data
df.dropna(subset=['text', 'sentiment'], inplace=True)


In [6]:
df

,text,sentiment
0,deserved candidate promoted promptly unbiased ...,1.0
1,got lot learning platform monthly learning pla...,1.0
2,based business unit get experience company pol...,1.0
3,client project good use latest tech work,1.0
4,worked ibm year tc year rd company year comple...,1.0
...,...,...
54021,le visibility next level promotion point time ...,0.0
54022,amazon adopted giving responsibility workplace...,0.0
54023,terrible experience working amazon pay leaders...,0.0
54024,logistics supply management application operat...,0.0


## preparation for sentiment analysis, consists:
- encode labels
- split data
- tokenize the text column on train and val

In [7]:
# Encode sentiment labels
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [8]:
# Split data into train and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2)


In [19]:
# GPU check available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [17]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 classes: negative and positive
model.to(device) # using gpu cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
# Tokenize data
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128)
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=128)


In [11]:
# Create torch dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

## fine tuned, results, and save.

The fine tuned consists:
- epoch: 3
-  bath size: 32

In [30]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    learning_rate=3e-5
)

In [31]:
class CustomTrainer(Trainer):
    def _prepare_inputs(self, inputs):
        # Move inputs to the right device
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
        return inputs

In [32]:
# Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [33]:
# Train model
trainer.train()

  2%|▏         | 79/3483 [18:31<13:17:53, 14.06s/it]
                                                   
  2%|▏         | 76/3483 [03:09<1:20:28,  1.42s/it]

{'loss': 0.6039, 'grad_norm': 7.995917797088623, 'learning_rate': 3e-06, 'epoch': 0.04}


                                                   
  2%|▏         | 76/3483 [04:08<1:20:28,  1.42s/it] 

{'loss': 0.585, 'grad_norm': 6.20795202255249, 'learning_rate': 6e-06, 'epoch': 0.09}


                                                   
  2%|▏         | 76/3483 [05:07<1:20:28,  1.42s/it] 

...

                                                     
  2%|▏         | 76/3483 [1:06:47<1:20:28,  1.42s/it]

{'loss': 0.5495, 'grad_norm': 2.4409220218658447, 'learning_rate': 3.3188065705665436e-07, 'epoch': 2.97}


                                                     
100%|██████████| 3483/3483 [1:05:15<00:00,  1.12s/it]

{'train_runtime': 3915.6233, 'train_samples_per_second': 28.449, 'train_steps_per_second': 0.89, 'train_loss': 0.5836303731746931, 'epoch': 3.0}


TrainOutput(global_step=3483, training_loss=0.5836303731746931, metrics={'train_runtime': 3915.6233, 'train_samples_per_second': 28.449, 'train_steps_per_second': 0.89, 'total_flos': 7327379780720640.0, 'train_loss': 0.5836303731746931, 'epoch': 3.0})

In [34]:
# Evaluate model
results = trainer.evaluate()
print(results)

100%|██████████| 291/291 [01:25<00:00,  3.39it/s]

{'eval_loss': 0.619106650352478, 'eval_runtime': 86.0218, 'eval_samples_per_second': 107.926, 'eval_steps_per_second': 3.383, 'epoch': 3.0}


In [35]:
# Save model pretrained model for hugging face
output_dir="D:\Project\Final Project Indonesia AI/02_11062024_bert_sentiment_model"
model.save_pretrained(output_dir, from_pt=True)
tokenizer.save_pretrained(output_dir)

('D:\\Project\\Final Project Indonesia AI/02_11062024_bert_sentiment_model\\tokenizer_config.json',
 'D:\\Project\\Final Project Indonesia AI/02_11062024_bert_sentiment_model\\special_tokens_map.json',
 'D:\\Project\\Final Project Indonesia AI/02_11062024_bert_sentiment_model\\vocab.txt',
 'D:\\Project\\Final Project Indonesia AI/02_11062024_bert_sentiment_model\\added_tokens.json')

In [41]:
# import matplotlib.pyplot as plt

# # Assuming `trainer` is your Trainer object with training history
# train_loss = trainer.state.log_history['loss']
# eval_loss = trainer.state.log_history['eval_loss']

# # Plot loss over epochs
# plt.plot(range(len(train_loss)), train_loss, label='Train Loss')
# plt.plot(range(len(eval_loss)), eval_loss, label='Eval Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Evaluation Loss')
# plt.legend()
# plt.show()

TypeError: list indices must be integers or slices, not str

## testing the fine tuned models with cleaned dataset

In [3]:
# Load tokenizer and fine-tuned model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("02_11062024_bert_sentiment_model") # the fine tuned model save

# Function to tokenize text and obtain BERT embeddings
def tokenize_and_predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits
    return predictions

# Function to compute percentage
def compute_percentage(predictions):
    softmax_output = torch.softmax(predictions, dim=1)
    positive_percentage = softmax_output[:, 1].item() * 100
    negative_percentage = 100 - positive_percentage
    return positive_percentage, negative_percentage

In [4]:
# User input
user_text = ("🤯jobdesk")

# Tokenize user input and get predictions
user_predictions = tokenize_and_predict(user_text)
positive_percentage, negative_percentage = compute_percentage(user_predictions)

# Determine sentiment
sentiment = "Positive" if user_predictions.argmax() == 1 else "Negative"

print("Sentiment:", sentiment)
print("Positive sentiment percentage:", positive_percentage)
print("Negative sentiment percentage:", negative_percentage)

Sentiment: Negative
Positive sentiment percentage: 40.34416675567627
Negative sentiment percentage: 59.65583324432373
